In [1]:
!python -m pip install --user --upgrade pip
!pip install --user keras
!pip install --user tensorflow
!pip install --user -U numpy
!pip install --user hyperas

Requirement already up-to-date: pip in c:\users\wangmi94\appdata\roaming\python\python36\site-packages (19.2.2)
Requirement already up-to-date: numpy in c:\users\wangmi94\appdata\roaming\python\python36\site-packages (1.17.0)


In [ ]:
#!/usr/bin/python
#-*- coding: UTF-8 -*-

In [ ]:
# Use CPU only
# Only Macbook needs to run this cell
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dense, LeakyReLU
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
import matplotlib.pyplot as plt
#from keras.layers.normalization import BatchNormalization
from keras import backend as K
import sklearn.metrics as skm, math
from keras.layers.advanced_activations import PReLU, ELU
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\wangmi94\AppData\R

## Load data and do data normalization

In [ ]:
# df = pd.read_csv('Options_R_new.csv')
# df = df.dropna()
# df.head()

In [3]:
def data():
    df = pd.read_csv('Options_R_new.csv')
    df = df.dropna()
    
    # Remove the option data has less than 7 days to maturity.
    df = df[df.maturity > 6]

    # Remove the option data with deep in-the-money and deep out-of-money
    def moneyness(S,X):
        return S/X
    
    df['moneyness'] = df.apply(lambda row: moneyness(row['underlying_price'], row['strike_price']), axis = 1)
    df = df[df.moneyness >= 0.7]
    df = df[df.moneyness <= 1.2]
    
    def intrinsic(S, K, C, P):
        return C*(max(S-K, 0)) + P*(max(K-S, 0))
    
    df['intrinsic'] = df.apply(lambda row: intrinsic(row['underlying_price'], row['strike_price'], row['cp_flag_C'], row['cp_flag_P']), axis = 1)
    
    sample_df = df.sample(n = 50000)
    
    y = sample_df['best_offer'].values
    X = sample_df[['maturity', 'strike_price', 'impl_volatility', 'underlying_price', 'cp_flag_C', 'cp_flag_P', 'interest_rate']]
    
    # Normalize the data to build a regression neural network model
    X = preprocessing.normalize(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    return X_train, y_train, X_test, y_test

## Build a keras Sequential model

In [1]:
# Because it is a simple regression problem, we should custom metrics function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis = -1))

def r_square(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

def mpe(y_true, y_pred):
    errors = K.sqrt(K.mean((y_true-y_pred)**2))/K.mean(y_true)
    return errors

def exp(x):
    return K.exp(x)

# Enable early stopping based on the loss of validation data
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=50)

NameError: name 'EarlyStopping' is not defined

In [5]:
def model(X_train, y_train, X_test, y_test):
    
#     def rmse(y_true, y_pred):
#         return K.sqrt(K.mean(K.square(y_pred - y_true), axis = -1))

    def r_square(y_true, y_pred):
        SS_res =  K.sum(K.square(y_true - y_pred)) 
        SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
        return (1 - SS_res/(SS_tot + K.epsilon()))

#     def mpe(y_true, y_pred):
#         errors = K.sqrt(K.mean((y_true-y_pred)**2))/K.mean(y_true)
#         return errors

    def exp(x):
        return K.exp(x)
    
    model = Sequential()
    model.add(Dense(32, input_shape = (7,)))
    model.add(LeakyReLU(alpha = 0.1))
    
    model.add(Dense({{choice([32, 64, 128, 256])}}))
    model.add(Activation('relu'))
    
    model.add(Dense({{choice([64, 128, 256])}}))
    model.add(Activation({{choice([ELU(), 'relu'])}}))
        
    model.add(Dense(1, activation = exp))
    
    es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=25)
        
    model.compile(loss = 'mse', metrics = [r_square], optimizer = {{choice(['rmsprop', 'adam'])}})
    
    result = model.fit(X_train, y_train,
                       batch_size = {{choice([64, 128, 256])}},
                       nb_epoch = 200,
                       verbose = 2,
                       validation_split = 0.1,
                       callbacks = [es])
    validation_acc = np.amax(result.history['val_r_square'])
    
    print('Best validation accuracy of epoch:', validation_acc)
    
    return {'loss': validation_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
a = 'OPPT'
print(type(a))

# str------>bytes(gbk)
b_utf8 = a.encode("charmap")
print(b_utf8)

d_unicode = b_utf8.decode("charmap")
print(d_unicode)

In [10]:
best_run, best_model = optim.minimize(model = model,
                                      data = data,
                                      algo=tpe.suggest,
                                      max_evals = 30,
                                      trials = Trials(),
                                      notebook_name = 'Option_Pricing_Parameters_tuning')
X_train, y_train, X_test, y_test = data()

>>> Imports:
#coding=utf-8

try:
    import os
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Activation, Dense, LeakyReLU
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import sklearn.metrics, math as skm
except:
    pass

try:
    from keras.layers.advanced_activations import PReLU, ELU
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import talos
e

W0817 13:18:07.503515 16768 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



  0%|                                                                             | 0/30 [00:00<?, ?it/s, best loss: ?]

W0817 13:18:07.541440 16768 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 13:18:07.545443 16768 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0817 13:18:07.586474 16768 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

C:\Users\wangmi94\research\temp_model.py:165: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.

W0817 13:18:07.718962 16768 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\keras\backend

Train on 36000 samples, validate on 4000 samples                                                                       
Epoch 1/200                                                                                                            
 - 0s - loss: 94.0223 - r_square: 0.0340 - val_loss: 86.5582 - val_r_square: 0.0310                                    

Epoch 2/200                                                                                                            
 - 0s - loss: 90.0170 - r_square: 0.0691 - val_loss: 83.5670 - val_r_square: 0.0752                                    

Epoch 3/200                                                                                                            
 - 0s - loss: 85.5462 - r_square: 0.0982 - val_loss: 78.2101 - val_r_square: 0.1234                                    

Epoch 4/200                                                                                                            
 - 0s - loss: 81.2848 - r_square: 0.1

...

 - 0s - loss: 68.5791 - r_square: 0.2849 - val_loss: 66.7329 - val_r_square: 0.2587                                    

Epoch 15/200                                                                                                           
 - 0s - loss: 67.2007 - r_square: 0.2912 - val_loss: 58.1921 - val_r_square: 0.3475                                    

Epoch 16/200                                                                                                           
 - 0s - loss: 64.6919 - r_square: 0.3248 - val_loss: 56.9473 - val_r_square: 0.3625                                    

Epoch 17/200                                                                                                           
 - 0s - loss: 63.6373 - r_square: 0.3348 - val_loss: 54.3302 - val_r_square: 0.3894                                    

Epoch 18/200                                                                                                           
 - 0s - loss: 61.6363 - r_square: 0.

 - 0s - loss: 7.7691 - r_square: 0.9197 - val_loss: 2.8254 - val_r_square: 0.9681                                      

Epoch 82/200                                                                                                           
 - 0s - loss: 7.7993 - r_square: 0.9183 - val_loss: 8.8217 - val_r_square: 0.9031                                      

Epoch 83/200                                                                                                           
 - 0s - loss: 8.0982 - r_square: 0.9162 - val_loss: 6.9721 - val_r_square: 0.9223                                      

Epoch 84/200                                                                                                           
 - 0s - loss: 7.6155 - r_square: 0.9208 - val_loss: 11.8966 - val_r_square: 0.8682                                     

Epoch 85/200                                                                                                           
 - 0s - loss: 7.9031 - r_square: 0.9

 - 0s - loss: 4.3798 - r_square: 0.9535 - val_loss: 1.5441 - val_r_square: 0.9825                                      

Epoch 149/200                                                                                                          
 - 0s - loss: 4.1506 - r_square: 0.9562 - val_loss: 1.5512 - val_r_square: 0.9824                                      

Epoch 150/200                                                                                                          
 - 0s - loss: 4.2560 - r_square: 0.9552 - val_loss: 2.9035 - val_r_square: 0.9677                                      

Epoch 151/200                                                                                                          
 - 0s - loss: 4.1061 - r_square: 0.9567 - val_loss: 2.1965 - val_r_square: 0.9750                                      

Epoch 152/200                                                                                                          
 - 0s - loss: 4.1690 - r_square: 0.9

Epoch 50/200                                                                                                           
 - 0s - loss: 20.9872 - r_square: 0.7860 - val_loss: 18.4022 - val_r_square: 0.7932                                    

Epoch 51/200                                                                                                           
 - 0s - loss: 20.1513 - r_square: 0.7902 - val_loss: 17.2117 - val_r_square: 0.8055                                    

Epoch 52/200                                                                                                           
 - 0s - loss: 20.5689 - r_square: 0.7875 - val_loss: 25.6473 - val_r_square: 0.7175                                    

Epoch 53/200                                                                                                           
 - 0s - loss: 19.9503 - r_square: 0.7918 - val_loss: 11.4858 - val_r_square: 0.8706                                    

Epoch 54/200                        

Epoch 117/200                                                                                                          
 - 0s - loss: 7.7716 - r_square: 0.9230 - val_loss: 3.4834 - val_r_square: 0.9605                                      

Epoch 118/200                                                                                                          
 - 0s - loss: 7.9359 - r_square: 0.9182 - val_loss: 6.0240 - val_r_square: 0.9339                                      

Epoch 119/200                                                                                                          
 - 0s - loss: 7.8547 - r_square: 0.9203 - val_loss: 4.1540 - val_r_square: 0.9541                                      

Epoch 120/200                                                                                                          
 - 0s - loss: 7.7594 - r_square: 0.9217 - val_loss: 11.2591 - val_r_square: 0.8737                                     

Epoch 121/200                       

In [11]:
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

Evalutation of best performing model:
10000/10000 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 23us/step
[8.590908522033692, 0.8950495520591736]
Best performing model chosen hyper-parameters:
{'Activation': 0, 'Dense': 0, 'Dense_1': 1, 'Dense_2': 2, 'optimizer': 0}


In [12]:
y_pred = best_model.predict(X_test)
print(y_pred[:10])
print(y_test[:10])

[[ 2.8370142 ]
 [ 0.04040735]
 [14.696942  ]
 [ 0.22345762]
 [ 0.4586056 ]
 [13.624421  ]
 [ 0.6908643 ]
 [ 0.31901586]
 [ 0.07384801]
 [ 0.0264882 ]]
[ 4.2   0.04 19.35  0.18  0.26 17.2   0.15  0.59  0.04  0.15]


In [13]:
best_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_117 (Dense)            (None, 32)                256       
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 32)                0         
_________________________________________________________________
dense_118 (Dense)            (None, 32)                1056      
_________________________________________________________________
activation_59 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_119 (Dense)            (None, 128)               4224      
_________________________________________________________________
activation_60 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_120 (Dense)            (None, 1)                 129       
Total para

In [14]:
def mean_percent_error(y_true, y_pred):
    errors = np.sqrt(np.mean((y_true-y_pred)**2))/np.mean(y_true)
    return errors

In [15]:
print("Average precentage pricing error: %f" % mean_percent_error(y_test, y_pred))
print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y_test, y_pred))
print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y_test, y_pred))
print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y_test, y_pred)))
print("R square (R^2):                 %f" % skm.r2_score(y_test, y_pred))

Average precentage pricing error: 2.160239
Mean absolute error (MAE):      1.542325
Mean squared error (MSE):       8.590909
Root mean squared error (RMSE): 2.931025
R square (R^2):                 0.913211


In [20]:
#To satisfy the req of talos, scipy must be in version 1.2 and above
#Run to check your version/ download
!pip install --user scipy

In [16]:
!pip install --user talos

  Created wheel for talos: filename=talos-0.5.0-cp36-none-any.whl size=41550 sha256=85533b326bb1d584789705865fbd9aef6104348ab5c935f13d861a02a045311c
  Stored in directory: C:\Users\wangmi94\AppData\Local\pip\Cache\wheels\fc\19\4a\5e72a9d4c2015760295561268d22f60c01c7cc599faf560ea3
  Created wheel for astetik: filename=astetik-1.9.8-cp36-none-any.whl size=56073 sha256=90f1732a38b5bac116be728e512ceca36cce2f7bf7fce800936186a7b3583079
  Stored in directory: C:\Users\wangmi94\AppData\Local\pip\Cache\wheels\ba\cc\e9\11c6a853d8379f295e17b68f2139ea1bbcd13c5b260822abc7
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1324 sha256=ed5dd2cc192f784a1b9f1fe79a391acedcc02b203088494f3d26f26815c349d9
  Stored in directory: C:\Users\wangmi94\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
  Created wheel for chances: filename=chances-0.1.6-cp36-none-any.whl size=52555 sha256=6db9e6623a50f7ba78eaad3e5af9c83dbc1f1ea0a65495e93c44914bcc590

ERROR: wrangle 0.6.7 has requirement scipy==1.2, but you'll have scipy 1.1.0 which is incompatible.


In [21]:
import talos

C:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [25]:
df = pd.read_csv('Options_R_new.csv')
df = df.dropna()
    
# Remove the option data has less than 7 days to maturity.
df = df[df.maturity > 6]

# Remove the option data with deep in-the-money and deep out-of-money
def moneyness(S,X):
    return S/X
    
df['moneyness'] = df.apply(lambda row: moneyness(row['underlying_price'], row['strike_price']), axis = 1)
df = df[df.moneyness >= 0.7]
df = df[df.moneyness <= 1.2]
    
def intrinsic(S, K, C, P):
    return C*(max(S-K, 0)) + P*(max(K-S, 0))
    
df['intrinsic'] = df.apply(lambda row: intrinsic(row['underlying_price'], row['strike_price'], row['cp_flag_C'], row['cp_flag_P']), axis = 1)
    
sample_df = df.sample(n = 500000)
    
y = sample_df['best_offer'].values
X = sample_df[['maturity', 'strike_price', 'impl_volatility', 'underlying_price', 'cp_flag_C', 'cp_flag_P', 'interest_rate']]
    
# Normalize the data to build a regression neural network model
X = preprocessing.normalize(X)

In [26]:
X.shape

(500000, 7)

In [30]:
def talos_model(X_train, y_train, X_val, y_val, params):
    model = Sequential()
    model.add(Dense(32, input_shape=(7,), activation=params['activation'], kernel_initializer='normal'))

    model.add(Dense(params['second_neuron'], activation = params['activation'], kernel_initializer = 'normal'))
    
    model.add(Dense(params['third_neuron'], activation = params['activation'], kernel_initializer = 'normal'))
    
    model.add(Dense(1, activation=params['last_activation'], kernel_initializer='normal'))
    
    model.compile(loss=params['losses'],
                  # here we add a regulizer normalization function from Talos
                  optimizer=params['optimizer'], #(lr=lr_normalizer(params['lr'],params['optimizer'])),
                  metrics=[r_square, mpe])
    
    out = model.fit(X_train, y_train,
                    validation_data = [X_val, y_val],
                    batch_size = params['batch_size'],
                    nb_epoch = params['epochs'],
                    verbose = 2)
    return out, model

In [31]:
p = {#'lr': (0.5, 5, 10),
     'second_neuron':[32, 64, 128, 256],
     'third_neuron':[64, 128, 256],
     'batch_size': [128, 256],
     'epochs': [150],
     #'dropout': (0, 0.5, 5),
     #'weight_regulizer':[None],
     #'emb_output_dims': [None],
     #'shape':['brick','long_funnel'],
     'optimizer': ['Adam', 'Nadam', 'RMSprop'],
     'losses': ['mse'],
     'activation':['relu', 'elu'],
     'last_activation': ['linear', exp]}

In [32]:
t = talos.Scan(x=X, y=y, params = p, model=talos_model, grid_downsample=0.1)


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

Train on 350000 samples, validate on 150000 samples
Epoch 1/150
 - 8s - loss: 84.8278 - r_square: 0.1016 - mpe: 1.5205 - val_loss: 45.2646 - val_r_square: 0.4947 - val_mpe: 1.1336
Epoch 2/150
 - 6s - loss: 51.0837 - r_square: 0.4378 - mpe: 1.1745 - val_loss: 34.3077 - val_r_square: 0.6095 - val_mpe: 0.9872
Epoch 3/150
 - 6s - loss: 34.1838 - r_square: 0.6102 - mpe: 0.9826 - val_loss: 74.2258 - val_r_square: 0.2306 - val_mpe: 1.4131
Epoch 4/150
 - 6s - loss: 132.6221 - r_square: -3.9248e-01 - mpe: 1.9133 - val_loss: 131.3850 - val_r_square: -3.9079e-01 - val_mpe: 1.9085
Epoch 5/150
 - 6s - loss: 132.4323 - r_square: -3.8928e-01 - mpe: 1.9115 - val_loss: 131.3850 - val_r_square: -3.9079e-01 - val_mpe: 1.9085
Epoch 6/150
 - 6s - loss: 132.4323 - r_square: -3.8931e-01 - mpe: 1.9120 - val_loss: 131.3850 - val_r_square: -3.9079e-01 - val_mpe: 1.9085
Epoch 7/150
 - 6s - loss: 7441297.9602 - r_square: -5.0623e+04 - mpe: 8.6090 - val_loss: 63.0352 - val_r_square: 0.2862 - val_mpe: 1.3430
Epoch 

Epoch 68/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 69/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 70/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 71/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 72/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 73/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 74/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 75/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 76/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 77/150
 - 6s 

Epoch 143/150
 - 5s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 144/150
 - 5s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 145/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 146/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 147/150
 - 5s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 148/150
 - 5s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 149/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf
Epoch 150/150
 - 6s - loss: nan - r_square: nan - mpe: inf - val_loss: nan - val_r_square: nan - val_mpe: inf



  4%|██▊                                                                             | 1/28 [14:39<6:35:51, 879.69s/it]

...

Epoch 130/150
 - 3s - loss: 0.3916 - r_square: 0.9956 - mpe: 0.1027 - val_loss: 0.2705 - val_r_square: 0.9969 - val_mpe: 0.0879
Epoch 131/150
 - 3s - loss: 0.3835 - r_square: 0.9957 - mpe: 0.1015 - val_loss: 0.3572 - val_r_square: 0.9959 - val_mpe: 0.1010
Epoch 132/150
 - 3s - loss: 0.3768 - r_square: 0.9957 - mpe: 0.1005 - val_loss: 0.2882 - val_r_square: 0.9967 - val_mpe: 0.0907
Epoch 133/150
 - 3s - loss: 0.3751 - r_square: 0.9957 - mpe: 0.1008 - val_loss: 0.6331 - val_r_square: 0.9931 - val_mpe: 0.1329
Epoch 134/150
 - 3s - loss: 0.3647 - r_square: 0.9959 - mpe: 0.0997 - val_loss: 0.6043 - val_r_square: 0.9932 - val_mpe: 0.1304
Epoch 135/150
 - 3s - loss: 0.4324 - r_square: 0.9951 - mpe: 0.1073 - val_loss: 0.4538 - val_r_square: 0.9948 - val_mpe: 0.1131
Epoch 136/150
 - 3s - loss: 0.3825 - r_square: 0.9957 - mpe: 0.1023 - val_loss: 0.3488 - val_r_square: 0.9960 - val_mpe: 0.0996
Epoch 137/150
 - 3s - loss: 0.3902 - r_square: 0.9956 - mpe: 0.1020 - val_loss: 0.5771 - val_r_square: 0


 96%|████████████████████████████████████████████████████████████████████████████▏  | 27/28 [3:01:54<06:41, 401.17s/it]

Train on 350000 samples, validate on 150000 samples
Epoch 1/150
 - 2s - loss: 70.5985 - r_square: 0.2597 - mpe: 1.3949 - val_loss: 47.3857 - val_r_square: 0.4908 - val_mpe: 1.1597
Epoch 2/150
 - 2s - loss: 25.5747 - r_square: 0.7279 - mpe: 0.8230 - val_loss: 11.0187 - val_r_square: 0.8819 - val_mpe: 0.5580
Epoch 3/150
 - 2s - loss: 12.0726 - r_square: 0.8711 - mpe: 0.5628 - val_loss: 60.9629 - val_r_square: 0.3300 - val_mpe: 1.3231
Epoch 4/150
 - 2s - loss: 7.8468 - r_square: 0.9162 - mpe: 0.4459 - val_loss: 20.3362 - val_r_square: 0.7780 - val_mpe: 0.7624
Epoch 5/150
 - 2s - loss: 5.8318 - r_square: 0.9380 - mpe: 0.3855 - val_loss: 3.4637 - val_r_square: 0.9630 - val_mpe: 0.3106
Epoch 6/150
 - 2s - loss: 4.9309 - r_square: 0.9479 - mpe: 0.3502 - val_loss: 18.9544 - val_r_square: 0.7922 - val_mpe: 0.7364
Epoch 7/150
 - 2s - loss: 3.6665 - r_square: 0.9609 - mpe: 0.3023 - val_loss: 7.1907 - val_r_square: 0.9227 - val_mpe: 0.4512
Epoch 8/150
 - 2s - loss: 3.1984 - r_square: 0.9661 - mpe:

Epoch 130/150
 - 2s - loss: 0.3476 - r_square: 0.9963 - mpe: 0.0955 - val_loss: 0.3619 - val_r_square: 0.9961 - val_mpe: 0.1008
Epoch 131/150
 - 2s - loss: 0.3500 - r_square: 0.9962 - mpe: 0.0958 - val_loss: 0.2628 - val_r_square: 0.9971 - val_mpe: 0.0865
Epoch 132/150
 - 2s - loss: 0.3340 - r_square: 0.9964 - mpe: 0.0940 - val_loss: 0.2285 - val_r_square: 0.9975 - val_mpe: 0.0808
Epoch 133/150
 - 2s - loss: 0.3464 - r_square: 0.9963 - mpe: 0.0947 - val_loss: 0.2186 - val_r_square: 0.9976 - val_mpe: 0.0789
Epoch 134/150
 - 2s - loss: 0.3411 - r_square: 0.9963 - mpe: 0.0953 - val_loss: 0.3752 - val_r_square: 0.9959 - val_mpe: 0.1032
Epoch 135/150
 - 2s - loss: 0.3417 - r_square: 0.9963 - mpe: 0.0946 - val_loss: 1.0161 - val_r_square: 0.9889 - val_mpe: 0.1696
Epoch 136/150
 - 2s - loss: 0.3313 - r_square: 0.9964 - mpe: 0.0938 - val_loss: 0.2434 - val_r_square: 0.9973 - val_mpe: 0.0833
Epoch 137/150
 - 2s - loss: 0.3330 - r_square: 0.9964 - mpe: 0.0946 - val_loss: 0.3842 - val_r_square: 0


100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [3:07:08<00:00, 375.08s/it]

In [33]:
# accessing the results data frame
t.data.head(28)

,round_epochs,val_loss,val_r_square,val_mpe,loss,r_square,mpe,second_neuron,third_neuron,batch_size,epochs,optimizer,losses,activation,last_activation
0,150,2.696853,-0.390793,0.277347,4.810371,-50623.028523,0.351555,128,256,128,150,Nadam,mse,elu,<function exp at 0x00000210F3F00598>
1,150,0.194180,0.422118,0.073775,0.275412,0.267278,0.086326,32,128,128,150,Adam,mse,relu,<function exp at 0x00000210F3F00598>
2,150,0.299667,0.518643,0.092411,0.400977,0.217137,0.104144,32,128,256,150,Nadam,mse,relu,linear
3,150,0.100013,0.711555,0.053253,0.150834,0.285959,0.063214,64,256,128,150,Nadam,mse,elu,linear
4,150,0.427885,0.147741,0.109800,1.033847,0.141781,0.161897,256,128,256,150,RMSprop,mse,elu,<function exp at 0x00000210F3F00598>
5,150,0.138474,-0.378176,0.062615,0.222749,-0.269721,0.075597,256,64,256,150,Nadam,mse,elu,<function exp at 0x00000210F3F00598>
6,150,0.273904,0.322291,0.087780,0.489298,0.125287,0.113626,32,128,128,150,RMSprop,mse,relu,linear
7,150,0.545549,0.071443,0.124707,1.305087,0.121921,0.183420,32,64,256,150,RMSprop,mse,elu,<function exp at 0x00000210F3F00598>
8,150,0.110299,0.243893,0.056030,0.159214,0.149105,0.064934,128,128,128,150,Nadam,mse,elu,linear
9,150,0.382262,-1.276186,0.104066,0.886513,0.126253,0.150055,256,256,256,150,RMSprop,mse,elu,<function exp at 0x00000210F3F00598>


In [34]:
t.peak_epochs_df

,val_loss,val_r_square,val_mpe,loss,r_square,mpe,acc_epoch,loss_epoch
1,23,30,23,23,6,23,NaN,NaN
2,137,0,137,148,0,148,NaN,0.076082
3,146,1,146,146,0,149,NaN,0.287665
4,133,1,133,148,0,148,NaN,0.317775
5,140,7,140,149,0,149,NaN,0.149815
6,108,0,108,144,1,145,NaN,0.319836
7,144,0,144,149,0,149,NaN,0.243461
8,134,0,134,149,0,149,NaN,0.092237
9,133,0,133,148,0,147,NaN,0.944993
10,145,5,145,147,0,146,NaN,0.213588


In [35]:
from talos.utils.best_model import best_model
from talos.utils.best_model import activate_model
model_id = best_model(t, 'val_loss', True)
model = activate_model(t, model_id)

In [36]:
y_pred = model.predict(X)

In [37]:
y_pred[:10]

array([[ 0.88117754],
       [ 0.6752697 ],
       [15.020019  ],
       [19.003056  ],
       [ 0.5785471 ],
       [ 4.60103   ],
       [17.027061  ],
       [34.845306  ],
       [ 0.11160436],
       [ 0.25077882]], dtype=float32)

In [38]:
y[:10]

array([ 0.75,  0.46, 15.8 , 20.3 ,  0.48,  4.45, 17.65, 37.6 ,  0.04,
        0.12])

In [39]:
#print("Average precentage pricing error: %f" % mean_percent_error(y, y_pred))
print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y, y_pred))
print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y, y_pred))
print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y, y_pred)))
print("R square (R^2):                 %f" % skm.r2_score(y, y_pred))

Mean absolute error (MAE):      0.232545
Mean squared error (MSE):       0.134124
Root mean squared error (RMSE): 0.366229
R square (R^2):                 0.998614


In [40]:
# access the summary details
t.details

experiment_name           081719140300_
random_method          uniform_mersenne
reduction_method                   None
reduction_interval                   50
reduction_window                     20
grid_downsample                     0.1
reduction_threshold                 0.2
reduction_metric                val_acc
reduce_loss                       False
complete_time            08/17/19/17:10
x_shape                     (500000, 7)
y_shape                       (500000,)
dtype: object

In [41]:
talos.Deploy(t, 'option_price_1', 'r_square');

Deploy package option_price_1 have been saved.


In [42]:
model.save('tune_2.h5')

## Reference: 
&emsp;&emsp;What does 'Accuracy' mean in Regression? https://github.com/keras-team/keras/issues/7947 <br>
&emsp;&emsp;https://keras.io/metrics/<br>
&emsp;&emsp;Scale, Standardize, or Normalize with Scikit-Learn https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02 <br>
&emsp;&emsp;The Day my Computer Won the Nobel Prize (Neural Network Option Pricing)  https://medium.com/datadriveninvestor/the-day-my-computer-won-the-nobel-prize-neural-network-option-pricing-d29b4379f1d2 <br>